# Homework 1

In [9]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

#make chart interactive
import mplcursors

import time
from datetime import date
import pytz

utc = pytz.UTC

## Question 1: [Macro] Average growth of GDP in 2023 

What is the average growth of GDP (in %) in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). 

Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [10]:
end = date.today()
print(f'Year = {end.year}, Month = {end.month}, Day = {end.day},')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Year = {end.year-70}, Month = {end.month}, Day = {end.day},')

print(f'Period for indexes = {start} to {end}')

Year = 2024, Month = 4, Day = 24,
Year = 1954, Month = 4, Day = 24,
Period for indexes = 1954-04-24 to 2024-04-24


In [12]:
gdpreal = pdr.DataReader('GDPC1', 'fred', start = start)
gdpreal.tail()

,GDPC1
DATE,
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [14]:
gdpreal['gdpc1_yoy'] = gdpreal['GDPC1'] / gdpreal['GDPC1'].shift(4)-1
gdpreal['gdpc1_qoq'] = gdpreal['GDPC1'] / gdpreal['GDPC1'].shift(1)-1
gdpreal.tail()

,GDPC1,gdpc1_yoy,gdpc1_qoq
DATE,,,
2022-10-01,21989.981,0.006517,0.006354
2023-01-01,22112.329,0.017179,0.005564
2023-04-01,22225.350,0.023825,0.005111
2023-07-01,22490.692,0.029269,0.011939
2023-10-01,22679.255,0.031345,0.008384


In [15]:
round(gdpreal[gdpreal.index.year == 2023]['gdpc1_yoy'].mean() *100,1)

2.5

## Question 2. [Macro] Inverse "Treasury Yield"
Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [16]:
dgs2 = pdr.DataReader('DGS2', 'fred', start='2000-01-01')
dgs10 = pdr.DataReader('DGS10', 'fred', start='2000-01-01')

df = dgs10.join(dgs2)
df['treasury_yield'] = df.DGS10 - df.DGS2
df.tail()

,DGS10,DGS2,treasury_yield
DATE,,,
2024-04-17,4.59,4.93,-0.34
2024-04-18,4.64,4.98,-0.34
2024-04-19,4.62,4.97,-0.35
2024-04-22,4.62,4.97,-0.35
2024-04-23,4.61,4.86,-0.25


In [17]:
df[df.treasury_yield == df.treasury_yield.min()]

,DGS10,DGS2,treasury_yield
DATE,,,
2023-07-03,3.86,4.94,-1.08


BONUS:

**What does Inverted yield curve mean for the market and investors?**

The inverted yield curve is significant for two main reasons:

1. **Historical Indicator of Recession** : Inverted yield curves have historically been a reliable predictor of economic recessions. When short-term interest rates rise above long-term rates, it often signals that investors expect weaker economic growth and lower inflation in the future. This expectation could lead to reduced business investment and consumer spending, eventually leading to a recession.
2. **Impact on Investor Sentiment** : Investors may interpret an inverted yield curve as a signal of economic uncertainty and potential market downturns. As a result, they may adjust their investment strategies, such as shifting investments away from stocks to safer assets like bonds or cash, in anticipation of a market downturn.

A long-term bond is a 10-Year U.S. Treasury bond. In an inverted yield curve situation, the yield on a 10-Year bond might be lower than the yield on a 2-Year bond. This suggests that investors are willing to accept lower returns on longer-term investments, reflecting expectations of slower economic growth or even recession in the future.

However, the yield curve can be distorted due to various factors, and while an inverted yield curve is often a reliable indicator of an impending recession, it may not always be foolproof. Here's why:

* **Central Bank Interventions** : Central banks, such as the Federal Reserve in the United States, can influence short-term interest rates through monetary policy actions, such as interest rate cuts or quantitative easing programs. These interventions can distort the yield curve by artificially suppressing short-term rates.
* **Market Sentiment and Demand** : Market participants' expectations, perceptions of risk, and demand for specific maturities can also distort the yield curve. For example, strong demand for longer-term bonds may drive down their yields, leading to flattening or inversion of the curve.

Therefore, while the inverted yield curve remains a valuable indicator, it should be considered alongside other economic data and factors when assessing the likelihood of a future recession.

### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [20]:
def calculate_growth(index, start, end):
    """
    Calculate the growth rate for a given ticker and start and end date.

    Parameters:
    index (str): Index name (e.g., '^MXX')
    time_frame (str): Time frame (e.g., '5y', '10y', '1y')

    Returns:
    float: Growth rate as a decimal
    """
    data = yf.download(tickers=index,
                       start = start,
                       end = end,
                       interval='1d',
                       progress=False)
    first_value = data['Close'].iloc[0]
    last_value = data['Close'].iloc[-1]
    growth_rate = (last_value / first_value) - 1
    return round(growth_rate ,2)

In [21]:
print(f'GSPC 5y growth: {calculate_growth("^GSPC", "2019-04-09", "2024-04-09")}%' )
print(f'IPC Mexico 5y growth: {calculate_growth("^MXX", "2019-04-09", "2024-04-09")}%' )

GSPC 5y growth: 0.81%
IPC Mexico 5y growth: 0.28%


#### Bonus:

Comapring growth for other indexes

In [22]:
def calculate_growth_bytime(index, time_frame):
    """
    Calculate the growth rate for a given ticker and time frame.

    Parameters:
    index (str): Index name (e.g., '^MXX')
    time_frame (str): Time frame (e.g., '5y', '10y', '1y')

    Returns:
    float: Growth rate as a decimal
    """
    data = yf.download(tickers=index,
                       period=time_frame,
                       interval='1d',
                       progress=False)
    first_value = data['Close'].iloc[0]
    last_value = data['Close'].iloc[-1]
    growth_rate = (last_value / first_value) - 1
    return round(growth_rate *100,2)

In [23]:
indexes = [
    "^GSPC",  # S&P 500
    "^IXIC",  # Nasdaq Composite
    "^DJI",   # Dow Jones Industrial Average
    "^FTSE",  # FTSE 100
    "^GDAXI", # DAX
    "^N225"   # Nikkei 225
]
time_frames = ['5y', '10y', '20y']

for t in time_frames:
  print("*"*20)
  for idx in indexes:
    print(f"{t} growth for {idx} : {calculate_growth_bytime(idx, t)}%")

********************
5y growth for ^GSPC : 73.32%
5y growth for ^IXIC : 93.54%
5y growth for ^DJI : 45.34%
5y growth for ^FTSE : 7.61%
5y growth for ^GDAXI : 46.91%
5y growth for ^N225 : 73.24%
********************
10y growth for ^GSPC : 172.17%
10y growth for ^IXIC : 285.54%
10y growth for ^DJI : 135.07%
10y growth for ^FTSE : 19.95%
10y growth for ^GDAXI : 89.44%
10y growth for ^N225 : 166.99%
********************
20y growth for ^GSPC : 346.63%
20y growth for ^IXIC : 671.45%
20y growth for ^DJI : 268.23%
20y growth for ^FTSE : 75.87%
20y growth for ^GDAXI : 338.43%
20y growth for ^N225 : 216.18%


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks
Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [24]:
def calculate_rangeRatio_52wks(ticker, start, end):
    """
    Calculate the 52 week range ratio of Adj. Close prices

    Parameters:
    ticker (str): Ticker/stock name (e.g., '^GOOG')
    start (str): start date
    end (str): end date

    Returns:
    float: 52 week range ratio of Adj. Close prices 
    """
    data = yf.download(tickers=ticker,
                       start = start,
                       end = end,
                       interval='1d',
                       progress=False)
    max = data['Adj Close'].max()
    min = data['Adj Close'].min()
    range = (max-min)/max

    return round(range, 2)


In [25]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']


range_ratios = {}
for ticker in tickers:
  range_ratios[ticker] = calculate_rangeRatio_52wks(ticker, start = '2023-01-01', end='2023-12-31')

sorted(range_ratios.items(), key=lambda x: x[1])

[('2222.SR', 0.21),
 ('BRK-B', 0.21),
 ('JPM', 0.28),
 ('AAPL', 0.37),
 ('GOOG', 0.39),
 ('MSFT', 0.42)]

BONUS:

**Why range ratio may be important for your research?**

The range ratio is an important metric for understanding the volatility of a stock's price. A higher range ratio indicates a larger difference between the highest and lowest prices, which can be a sign of higher volatility. This information can be useful for investors when evaluating the risk and potential returns of a stock.

For example, a stock with a high range ratio may be more attractive to investors who are willing to take on higher risk in exchange for the potential for higher returns. Conversely, a stock with a low range ratio may be more appealing to investors who are more risk-averse and prefer more stable returns.

Additionally, the range ratio can be used in conjunction with other technical analysis indicators to identify potential trading opportunities or to assess the overall market conditions. For instance, a stock with a high range ratio may be more susceptible to sudden price swings, which could present opportunities for short-term traders.

### Question 5. [Stocks] Dividend Yield

**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained  dollar 1.25 dividends paid and the end year stock price is dollar 100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [28]:
def calculate_dividends(ticker, start, end):
    """
    Calculate the dividend for a stock

    Parameters:
    ticker (str): Ticker/stock name (e.g., '^GOOG')
    start (str): start date
    end (str): end date

    Returns:
    float: 52 week range ratio of Adj. Close prices 
    """
    data = yf.download(tickers=ticker,
                   start=start,
                   end=end,
                   interval='1d',
                   actions=True,
                   progress=False)
    total_dividends = data['Dividends'].sum()
    last_adj_closing_price = data['Adj Close'].iloc[-1]

    divident_yield = total_dividends / last_adj_closing_price

    return round(divident_yield * 100, 2)

In [29]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

for t in tickers:
  print(f"The Divident yield for {t}: {calculate_dividends(t, start = '2023-01-01', end='2023-12-31')}%")

The Divident yield for 2222.SR: 2.77%
The Divident yield for BRK-B: 0.0%
The Divident yield for AAPL: 0.49%
The Divident yield for MSFT: 0.74%
The Divident yield for GOOG: 0.0%
The Divident yield for JPM: 2.41%
